## Pyfavor's Notebook front-end for creating eFavor input files

In [4]:
import pathlib
import wntr
import xlwt
import pandas as pd
from typing import List, Dict, Any, Literal
from pyfavor.utils import hr_to_sec, hr_to_min, read_table, find_os, install_package
from pyfavor.create import (
    create_times_df, create_inlets_df, create_flows_df, create_pressure_df, 
    get_logger_info, get_notes, write_efavor_pressures)
from pyfavor.simulate import simulate_network, get_junction_pressures, get_inlet_flows

## Define execution options

In [ ]:
use_config: bool = False

In [ ]:
if use_config:
    # Read config file
    network_config = read_table(pathlib.Path("config.yaml"))['network']
    experiment_config = read_table(pathlib.Path("config.yaml"))['experiment']
    defaults_config = read_table(pathlib.Path("config.yaml"))['default']
    # Read variables from config dictionary
    inlet_valve_name = network_config['inlet_valve']
    inlet_pipe_name = network_config['inlet_pipe']
    measurement_period_mins = experiment_config['measurement_period']
    step_period_mins = experiment_config['step_period']
    flow_setting = experiment_config['flow_setting']
    start_hr = experiment_config['start_hr']
    end_hr = experiment_config['end_hr']
    network_file = defaults_config['network_file']
    efavor_logger_file = defaults_config['efavor_logger_file']
    efavor_pressure_file = defaults_config['efavor_pressure_file']
else:
    inlet_valve_name = "Valve_1"
    inlet_pipe_name = "Pipe_1"
    measurement_period_mins = 15
    step_period_mins = 60
    flow_setting = "15min"
    start_hr = 3
    end_hr = 6
    network_file = "networks/St_Servais_Centre_leak.inp"
    efavor_logger_file = "efavor/e_FAVOR_St_Servais.xls"
    efavor_pressure_file = "efavor/e_FAVOR_St_Servais_updated_new.xlsx"

In [4]:
# Assert and calculate number of flow measurements in a pressure step
assert step_period_mins % measurement_period_mins == 0

In [5]:
n_measurements_in_step = step_period_mins / measurement_period_mins

## 1. Load information about logger IDs from eFavor logger placement module

In [6]:
logger_info = get_logger_info(efavor_logger_file)
notes = get_notes(efavor_logger_file)

junction_ids = logger_info['Nearest Node ID (EPANET Junction ID)'].to_list()
inlet_nodes = logger_info[logger_info["Is inlet node? (true/false)"] == True]
inlet_logger_ids = inlet_nodes['Logger ID'].to_list()
inlet_logger_junction_ids = inlet_nodes['Nearest Node ID (EPANET Junction ID)'].to_list()
# Create a mapping between logger IDs and EPANET Junction IDS
epanet_to_logger_id_map = \
    dict(zip(logger_info['Nearest Node ID (EPANET Junction ID)'], logger_info['Logger ID']))

## 2. Simulate the EPANET network with WNTR and extract pressures from selected nodes

In [7]:
results = simulate_network(network_file)

## 3. Create sheets for the spreadsheet Excel file

In [8]:
pressure_df = create_pressure_df(
    junction_ids, results, n_measurements_in_step,
    hr_to_sec(start_hr), hr_to_sec(end_hr),
    epanet_to_logger_id_map)

inlet_flows = get_inlet_flows(
    results, flow_setting, inlet_pipe_name,
    hr_to_sec(start_hr), hr_to_sec(end_hr))

inlets_df = create_inlets_df(
    inlet_junction_id = inlet_logger_junction_ids[0],
    id_to_junction_map = epanet_to_logger_id_map,
    valve_id = inlet_valve_name )

flows_df = create_flows_df(
    inlet_logger_id = inlet_logger_ids[0], 
    flows=inlet_flows,
    flow_setting=flow_setting,
    n_meas=n_measurements_in_step)

times_df = create_times_df(
    start_time = hr_to_min(start_hr), 
    measurement_time = measurement_period_mins)

## 4. Write to Excel

In [9]:
write_efavor_pressures(
    efavor_pressure_file, 
    logger_info, 
    inlets_df, 
    pressure_df, 
    flows_df, 
    times_df, 
    notes)